In [1]:
import pandas as pd
intents = pd.read_csv("/content/Bank_dataset.csv")
test = pd.read_csv("/content/bank_test.csv")
train = pd.read_csv("/content/bank_train.csv")

In [2]:
# Define column headings
headings = ['Text', 'Label']

# Assign column headings to the DataFrame
train.columns = headings
test.columns = headings
intents.columns = headings

In [3]:
import numpy as np
import pandas as pd
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
train.shape

(78, 2)

In [5]:
test.shape

(20, 2)

In [6]:
train.head(10)

,Text,Label
0,I OPENED A NEW ACCOUNT IN YOUR BANK BUT WHEN I...,1
1,I APPLIED FOR DEBIT OR CREDIT CARD WHEN I OPEN...,0
2,TILL NOW I DID NOT GET ANY DEBIT OR CREDIT CAR...,0
3,I GAVE AN APPLICATION FOR LINKING UNIQUE NUMBE...,1
4,I WANT TO CHANGE THE MOBILE NUMBER THAT WAS PR...,1
5,I LOST MY DEBIT OR CREDIT CARD YESTERDAY . SO ...,0
6,PLEASE BLOCK MY DEBIT OR CREDIT CARD BECAUSE I...,0
7,I GAVE AN APPLICATION FOR LINKING MY MOBILE NU...,1
8,I GAVE AN APPLICATION FOR LINKING MY MOBILE NU...,1
9,Transaction status is settled from my side bu...,2


In [8]:
train['Label'].value_counts()

Label
2    29
1    26
0    23
Name: count, dtype: int64

We perform one hot encoding on the target variable to convert into a matrix of 0s and 1s.

In [9]:
encode_target= oneHot().fit(np.array(train.Label).reshape(-1,1))

In [10]:
train_target_encoded= encode_target.transform(np.array(train.Label).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.Label).reshape(-1,1)).toarray()

In [11]:
train_target_encoded

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [12]:
test_target_encoded

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [14]:
train["Text"]= train.Text.map(lambda l: l.lower())
test["Text"]= test.Text.map(lambda l: l.lower())

In [15]:
train["Text"]= train.Text.map(word_tokenize)
test["Text"]= test.Text.map(word_tokenize)

In [16]:
#Output of the above exercise looks like this
train["Text"]

0     [i, opened, a, new, account, in, your, bank, b...
1     [i, applied, for, debit, or, credit, card, whe...
2     [till, now, i, did, not, get, any, debit, or, ...
3     [i, gave, an, application, for, linking, uniqu...
4     [i, want, to, change, the, mobile, number, tha...
                            ...                        
73    [i, want, to, transfer, money, from, my, accou...
74    [i, want, to, transfer, money, from, my, accou...
75    [i, made, a, transaction, with, my, debit, car...
76    [i, made, a, transaction, on, irctc, using, my...
77    [i, applied, for, emi, on, an, online, shoppin...
Name: Text, Length: 78, dtype: object

In [17]:
def clean_data_rm_stop(strings, stop_list):
    sw= [str for str in strings if str not in stop_list]
    return sw

stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

train["Text"]= train.Text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))
test["Text"]= test.Text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))

In [20]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["Text"]= train.Text.map(lambda s: [stem_func.stem(x) for x in s])
train["Text"]= train.Text.apply(normalize)

test["Text"]= test.Text.map(lambda s: [stem_func.stem(x) for x in s])
test["Text"]= test.Text.apply(normalize)

In [21]:
train.head()

,Text,Label
0,o p e n n e w a c c o u n t b a n k g ...,1
1,a p p l i d e b i t c r e d i t c a r d ...,0
2,t i l l g e t d e b i t c r e d i t c ...,0
3,g a v e a p p l i c l i n k u n i q u ...,1
4,w a n t c h a n g m o b i l n u m b e r ...,1


In [22]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.Text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.Text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.Text)

In [23]:
tokenized_train_data

[[6,
  14,
  4,
  3,
  3,
  4,
  19,
  1,
  5,
  5,
  6,
  11,
  3,
  2,
  15,
  1,
  3,
  20,
  18,
  6,
  2,
  14,
  1,
  10,
  10,
  15,
  6,
  6,
  20,
  21,
  6,
  11,
  3,
  12,
  13,
  8,
  12,
  12,
  9,
  3,
  1,
  13,
  4,
  19,
  7,
  6,
  3,
  18,
  9,
  8,
  14,
  7,
  8,
  3,
  2],
 [1,
  14,
  14,
  9,
  8,
  12,
  4,
  15,
  8,
  2,
  5,
  7,
  4,
  12,
  8,
  2,
  5,
  1,
  7,
  12,
  6,
  14,
  4,
  3,
  1,
  5,
  5,
  6,
  11,
  3,
  2,
  15,
  1,
  3,
  20,
  9,
  1,
  10,
  2,
  13,
  6,
  3,
  2,
  16,
  18,
  4,
  2,
  2,
  8,
  9,
  9],
 [2,
  8,
  9,
  9,
  18,
  4,
  2,
  12,
  4,
  15,
  8,
  2,
  5,
  7,
  4,
  12,
  8,
  2,
  5,
  1,
  7,
  12,
  1,
  14,
  14,
  9,
  8,
  9,
  1,
  10,
  2,
  13,
  6,
  3,
  2,
  16,
  15,
  1,
  3,
  20],
 [18,
  1,
  22,
  4,
  1,
  14,
  14,
  9,
  8,
  5,
  9,
  8,
  3,
  20,
  11,
  3,
  8,
  23,
  11,
  3,
  11,
  13,
  15,
  4,
  7,
  15,
  1,
  3,
  20,
  1,
  5,
  5,
  6,
  11,
  3,
  2,
  9,
  1,
  10,
  2,
  13,

In [24]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")

In [25]:
train_data

array([[ 8, 12, 12, ...,  8,  3,  2],
       [15,  1,  3, ...,  8,  9,  9],
       [ 7, 12,  1, ...,  1,  3, 20],
       ...,
       [ 4, 10, 10, ..., 11,  3,  2],
       [ 6,  3,  4, ..., 11,  3,  2],
       [ 1,  5,  5, ...,  8,  2,  4]], dtype=int32)

In [26]:
train_data.shape

(78, 20)

In [27]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [28]:
trans_matrix_train.shape

(78, 20, 28)

In [29]:
trans_matrix_test.shape

(20, 20, 28)

In [30]:
class lstm_model_class(object):
    def build_lstm_model(self,input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
      ip_layer= Input(shape= (num_steps, input_dimensions)) #Define embedded layer with shape as number of steps and input dimensions. Note that both these are input variables to the model.

      lstm_model= LSTM(units= num_steps)(ip_layer) #Make the LSTM layer with number of steps as memory units
      dense_layer_1= Dense(op_shape, kernel_initializer= he_uniform(), #he_uniform draws samples in uniform distribution with -inf to +inf as range.
                   bias_initializer= "zeros",
                   kernel_regularizer= l2(l= kernel_reg),
                   bias_regularizer= l2(l= bias_reg))(lstm_model) # Create the  Dense layer which is the regular deeply connected layer
      int_layer= BatchNormalization()(dense_layer_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      int_layer= relu(int_layer) #This applies the rectified linear unit activation function
      int_layer= Dropout(rate= dropout_rate)(int_layer) #Dropout is used to define Dropout layer that sets input units to 0 with a frequency. Here it is dropout_rate
      output_1= Dense(op_shape, kernel_initializer= glorot_uniform(), #glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units
             bias_initializer= "zeros",
             kernel_regularizer= l2(l= kernel_reg),
             bias_regularizer= l2(l= bias_reg))(dense_layer_1) # Create another dense layer which is the output of the model.
      output_1= BatchNormalization()(output_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      final_output= softmax(output_1, axis= 1)

      loss_func= cce() # Since it is a multi-class classification problem, categorical crossentropy(cce) is used as the loss function
      perf_metrics= AUC() #our performance metric will be area under the curve
      optimizer= Adam() #we shall use Adam optimizer as our optimizer
      self.final_model= Model(inputs= [ip_layer], outputs= [final_output]) #Build the model with input and output layers
      self.final_model.compile(optimizer= optimizer, loss= loss_func, metrics= [perf_metrics]) #Compiling the keras model

    def train_lstm_model(self,x, y, valid_split, ep):
      self.final_model.fit(x, y, validation_split= valid_split, epochs= ep) #Create the train model


    def predict_lstm_model(self,x):
      return self.final_model.predict(x)    #Create the predict model


In [31]:
steps= trans_matrix_train.shape[1] #Define the number of steps is usually the number of steps in the train data.
input_dim= trans_matrix_train.shape[2] #Input dimension. Number of unique words in the train data
output_shape= train_target_encoded.shape[1] #Output shape. Usually the same number as the number of classes in the target variable. Here we have 8.
final_model= lstm_model_class()
final_model.build_lstm_model(input_dimensions= input_dim,
                  op_shape= output_shape,
                  num_steps= steps,
                  dropout_rate= 0.5, # Meaning 1 in 2 inputs will be randomly executed.
                  bias_reg= 0.3, # Reduce the bias in the model
                  kernel_reg= 0.3) #Reduce the weights excluding bias.



In [33]:
final_model.train_lstm_model(trans_matrix_train, train_target_encoded,
           0.2, 30) #Model takes train data, train target variable, validation split(here it is 80:20) and number of epochs.

Epoch 1/30
2/2 [==============================] - 0s 183ms/step - loss: 1.9470 - auc: 0.9553 - val_loss: 2.4405 - val_auc: 0.5166
Epoch 2/30
2/2 [==============================] - 0s 87ms/step - loss: 1.9204 - auc: 0.9603 - val_loss: 2.4297 - val_auc: 0.5146
Epoch 3/30
2/2 [==============================] - 0s 81ms/step - loss: 1.9086 - auc: 0.9578 - val_loss: 2.4179 - val_auc: 0.5127
Epoch 4/30
2/2 [==============================] - 0s 94ms/step - loss: 1.8845 - auc: 0.9628 - val_loss: 2.4027 - val_auc: 0.5146
Epoch 5/30
2/2 [==============================] - 0s 134ms/step - loss: 1.8607 - auc: 0.9684 - val_loss: 2.3908 - val_auc: 0.5127
Epoch 6/30
2/2 [==============================] - 0s 91ms/step - loss: 1.8457 - auc: 0.9714 - val_loss: 2.3783 - val_auc: 0.5078
Epoch 7/30
2/2 [==============================] - 0s 88ms/step - loss: 1.8564 - auc: 0.9559 - val_loss: 2.3653 - val_auc: 0.5039
Epoch 8/30
2/2 [==============================] - 0s 85ms/step - loss: 1.8187 - auc: 0.9680 - v

In [35]:
pred_train= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_train)) #Predict on the train matrix and look at the performance
print(classification_report(train.Label, pred_train)) #Print the classification report

3/3 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

           0       0.94      0.70      0.80        23
           1       0.60      0.92      0.73        26
           2       0.71      0.52      0.60        29

    accuracy                           0.71        78
   macro avg       0.75      0.71      0.71        78
weighted avg       0.74      0.71      0.70        78



In [36]:
pred_test= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_test)) #Predict on the test data
print(classification_report(test.Label, pred_test)) #Print the classification report

1/1 [==============================] - 0s 62ms/step
              precision    recall  f1-score   support

           0       1.00      0.38      0.55         8
           1       0.36      0.80      0.50         5
           2       0.33      0.29      0.31         7

    accuracy                           0.45        20
   macro avg       0.57      0.49      0.45        20
weighted avg       0.61      0.45      0.45        20

